In [1]:
import duckdb
import ibis
from ibis import _


In [2]:
import os

# Check if the file exists, if not, create it
if not os.path.exists('./db/sonotheque.duckdb'):
    # Create an empty DuckDB file
    duckdb.connect('./db/sonotheque.duckdb').close()
    print("Created sonotheque.duckdb file.")
else:
    print("sonotheque.duckdb file already exists.")

# Create a DuckDB connection
con = duckdb.connect('./db/sonotheque.duckdb')
con.install_extension("spatial")
con.load_extension("spatial")


sonotheque.duckdb file already exists.


In [2]:
con = duckdb.connect('./db/sonotheque.duckdb')

In [ ]:
# Aggregate records based on SON_PK and return first NOM_SCIENTIFIQUE
result = con.execute("""
    SELECT 
        SON_PK,
        FIRST(SON_UID) AS SON_UID,
        FIRST(COLLECTION_CODE) AS COLLECTION_CODE,
        FIRST(NOM_SCIENTIFIQUE) AS NOM_SCIENTIFIQUE
    FROM records
    GROUP BY SON_PK
""").fetchall()

# Display the first few results
print("Aggregated records (first 50):")
for row in result[:50]:
    print(f"SON_PK: {row[0]}, SON_UID: {row[1]}, COLLECTION_CODE: {row[2]}, NOM_SCIENTIFIQUE: {row[3]}")

# Get the total count of unique SON_PK
unique_count = con.execute("SELECT COUNT(DISTINCT SON_PK) FROM records").fetchone()[0]
print(f"\nTotal number of unique SON_PK: {unique_count}")


In [ ]:
# Aggregate records based on SON_PK and return first NOM_SCIENTIFIQUE
result = con.execute("""
    SELECT 
        NOM_SCIENTIFIQUE,
        FIRST(SON_UID) AS SON_UID,
        FIRST(COLLECTION_CODE) AS COLLECTION_CODE,
    FROM records
    GROUP BY NOM_SCIENTIFIQUE
""").fetchall()

# Display the first few results
print("Aggregated records (first 50):")
for row in result[:50]:
    print(f"NOM_SCIENTIFIQUE: {row[0]}, SON_UID: {row[1]}, COLLECTION_CODE: {row[2]}")




# Get the total count of unique SON_PK
unique_count = con.execute("SELECT COUNT(DISTINCT NOM_SCIENTIFIQUE) FROM records").fetchone()[0]
print(f"\nTotal number of unique SON_PK: {unique_count}")

In [14]:
con.sql("""
    SELECT 
        NOM_SCIENTIFIQUE,
        FIRST(SON_UID) AS SON_UID,
        FIRST(COLLECTION_CODE) AS COLLECTION_CODE,
    FROM records
    GROUP BY NOM_SCIENTIFIQUE
""").show()

┌──────────────────────────────────────────────────┬───────────────────────────────┬─────────────────┐
│                 NOM_SCIENTIFIQUE                 │            SON_UID            │ COLLECTION_CODE │
│                     varchar                      │            varchar            │     varchar     │
├──────────────────────────────────────────────────┼───────────────────────────────┼─────────────────┤
│ Hyla arborea (Linnaeus, 1758)                    │ 1423750549968saMd3IX4atyu8Osf │ SO              │
│ Gryllus campestris Linnaeus, 1758                │ 13867749528019ZTeQN2bXAkp7vVn │ SO              │
│ Notosciobia sp affinis paranola Otte, 1987       │ 1452140705499viYCyHQU70KGgIAU │ SO              │
│ Phonarellus minor (Chopard,1959)                 │ 1389780653418oIPwvLh4XYRHyr2V │ SO              │
│ Cardiodactylus kondoi Otte, 2007                 │ 1431533770611Ygcat0MITMCbsJRe │ SO              │
│ Acanthogryllus asiaticus Gorochov, 1990          │ 1467386703444ufUnTqI

In [ ]:
# con.sql("""
#     INSTALL spatial;
# LOAD spatial;
#     ALTER TABLE records
#     ADD COLUMN geometry GEOMETRY;

#     UPDATE records
#     SET geometry = ST_Point(LONGITUDE::FLOAT, LATITUDE::FLOAT)
#     WHERE LONGITUDE IS NOT NULL AND LATITUDE IS NOT NULL;
# """)


# Verify the new geometry column
con.sql("""
    SELECT 
        NOM_SCIENTIFIQUE,
        LATITUDE,
        LONGITUDE,
        ST_AsText(geometry) AS geometry_wkt
    FROM records
    WHERE geometry IS NOT NULL
    LIMIT 5
""").show()

print("\nTotal records with valid geometry:")


In [19]:
con.sql("SELECT COUNT(*) FROM records WHERE geometry IS NOT NULL").show()


┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│       202663 │
└──────────────┘



In [ ]:
# Aggregate records based on SON_PK and return first NOM_SCIENTIFIQUE
result = con.sql("""
    SELECT 
        uuid() AS UUID,
        NOM_SCIENTIFIQUE,
        LIST(SON_UID) AS SON_UIDS,
        FIRST(COLLECTION_CODE) AS COLLECTION_CODE,
    FROM records
    GROUP BY NOM_SCIENTIFIQUE
""")


In [ ]:
# results_df = con.sql("""
#     SELECT 
#         uuid() AS UUID,
#         NOM_SCIENTIFIQUE,
#         LIST(SON_UID) AS SON_UIDS,
#         FIRST(COLLECTION_CODE) AS COLLECTION_CODE,
#     FROM records
#     GROUP BY NOM_SCIENTIFIQUE
# """).df()

# results_df.head()


In [3]:
# Add a UUID column to records and generate a UUID for each unique SON_UID value
con.sql("""
    ALTER TABLE records
    ADD COLUMN record_uuid UUID;
""")

# Generate and assign UUIDs for each unique SON_UID
con.sql("""
    WITH unique_son_uids AS (
        SELECT DISTINCT SON_UID
        FROM records
    ),
    uuid_mapping AS (
        SELECT 
            SON_UID,
            uuid() AS new_uuid
        FROM unique_son_uids
    )
    UPDATE records
    SET record_uuid = uuid_mapping.new_uuid
    FROM uuid_mapping
    WHERE records.SON_UID = uuid_mapping.SON_UID;
""")

# Verify the new column and data
print("Sample of records with new UUID column:")
con.sql("""
    SELECT 
        record_uuid,
        SON_UID,
        NOM_SCIENTIFIQUE,
        COUNT(*) OVER (PARTITION BY SON_UID) as records_per_son_uid
    FROM records
    ORDER BY SON_UID
    LIMIT 10
""").show()


Sample of records with new UUID column:
┌──────────────────────┬───────────────────────────────┬─────────────────────────────────────────┬─────────────────────┐
│     record_uuid      │            SON_UID            │            NOM_SCIENTIFIQUE             │ records_per_son_uid │
│         uuid         │            varchar            │                 varchar                 │        int64        │
├──────────────────────┼───────────────────────────────┼─────────────────────────────────────────┼─────────────────────┤
│ c68e096d-49d0-416e…  │ 1379684303448THUEerYPSdH2oaAy │ Cicada barbara lusitanica Boulard, 1982 │                   6 │
│ c68e096d-49d0-416e…  │ 1379684303448THUEerYPSdH2oaAy │ Cicada barbara lusitanica Boulard, 1982 │                   6 │
│ c68e096d-49d0-416e…  │ 1379684303448THUEerYPSdH2oaAy │ Cicada barbara lusitanica Boulard, 1982 │                   6 │
│ c68e096d-49d0-416e…  │ 1379684303448THUEerYPSdH2oaAy │ Cicada barbara lusitanica Boulard, 1982 │               

In [4]:

print("\nTotal number of unique SON_UIDs:")
con.sql("SELECT COUNT(DISTINCT SON_UID) FROM records").show()

print("\nTotal number of records with UUID:")
con.sql("SELECT COUNT(DISTINCT record_uuid) FROM records").show()




Total number of unique SON_UIDs:
┌─────────────────────────┐
│ count(DISTINCT SON_UID) │
│          int64          │
├─────────────────────────┤
│                   30966 │
└─────────────────────────┘


Total number of records with UUID:
┌─────────────────────────────┐
│ count(DISTINCT record_uuid) │
│            int64            │
├─────────────────────────────┤
│                       30966 │
└─────────────────────────────┘



In [4]:
# Aggregate records based on SON_PK and return first NOM_SCIENTIFIQUE
results_df = con.sql("""
    SELECT 
        uuid() AS UUID,
        NOM_SCIENTIFIQUE,
        LIST(SON_UID) AS SON_UIDS,
        FIRST(COLLECTION_CODE) AS COLLECTION_CODE,
    FROM records
    GROUP BY NOM_SCIENTIFIQUE
""").df()

In [6]:
import ibis
from ibis import _

ibis.options.interactive = True

con = ibis.duckdb.connect('./db/sonotheque.duckdb')
con.list_tables()

['import_records', 'records']

In [8]:
con.load_extension("spatial")
records = con.table("records")


In [11]:
records.rename("snake_case")

┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ son_pk ┃ son_uid                       ┃ type   ┃ titre                                     ┃ institution_code ┃ collection_code ┃ catalog_number ┃ valide ┃ valide_par ┃ publie ┃ date_publication_debut ┃ date_publication_fin ┃ publication_scientifique                                                         ┃ recorder                        ┃ micro        ┃ parabole ┃ frequence_echantillonnage ┃ support_original ┃ qualite ┃ date_enregistrement_int ┃ heure_enregistrement_int ┃ lieu_enregistrement ┃ temperature ┃ commentaires_enregistrement ┃ contexte_comportemental          ┃ habitat ┃ description_habitat ┃ continent_ocean ┃ pays             ┃ localite                                             ┃ latitude  ┃ longitude ┃ altitude ┃ commentaires                                ┃ datecre           